In [9]:
import os
import numpy as np

import LMCE.cfusdlog as cfusdlog
import LMCE.uav_trajectory as uav_trajectory
from LMCE.error_estimation import error_calculator, find_best_cutoff

Here we compare the different average errors for each flight. The flights have a timescale of 0.6.

In [23]:
# Get desired path
traj = uav_trajectory.Trajectory()
traj.loadcsv("./LMCE/flight_paths/figure8.csv")
traj.stretchtime(2)

ts = np.arange(0, traj.duration, 0.01)
evals = np.empty((len(ts), 15))
for t, i in zip(ts, range(0, len(ts))):
    e = traj.eval(t)
    e.pos += np.array([0, 0, 1])
    evals[i, 0:3] = e.pos

target_pos = evals.transpose()

main_path = "./final_results/cf21/"
categories = ["lee", "nn", "standard", "tree"]


for c in categories:
    avg_errors = []
    avg_it = []
    path = main_path + c
    for filename in os.listdir(path):
        # if c == "nn" and filename == "nn_log05":
        #     continue

        data_path = path + "/" + filename
        # Get real path
        data = cfusdlog.decode(data_path)['fixedFrequency']
        real_pos = [data["stateEstimate.x"], data["stateEstimate.y"]]

        # Calculate error
        cutoff = find_best_cutoff(real_pos, target_pos)
        error = error_calculator(cutoff, real_pos, target_pos, vis=False)
        avg_errors.append(error)
        avg_it.append(sum(data["nn_perf.inf_tim"])/len(data["nn_perf.inf_tim"]))

    avg_error = sum(avg_errors)/len(avg_errors)
    avg_it = sum(avg_it)/len(avg_it)
    print("----------------------------------------")
    print(f"Avg. error for '{c}': {avg_error:.5f} (m) for the invidual flights: {avg_errors}")
    print(f"Avg. inf time for '{c}': {avg_it:.5f}")


----------------------------------------
Avg. error for 'lee': 0.13349 (m) for the invidual flights: [0.13265049481819421, 0.13694333235273984, 0.13339751793545943, 0.13115956600000897, 0.13531140583192675, 0.13263341601842876, 0.135313974345844, 0.13049556399950185]
Avg. inf time for 'lee': 0.00000
----------------------------------------
Avg. error for 'nn': 0.13381 (m) for the invidual flights: [0.13212407978684707, 0.13625985059970336, 0.13488776243200654, 0.13529046958570806, 0.13132636549520055, 0.13292686914900936, 0.13385421284089866]
Avg. inf time for 'nn': 3.69467
----------------------------------------
Avg. error for 'standard': 0.13100 (m) for the invidual flights: [0.12754159040825416, 0.1303852509048947, 0.13054595719351833, 0.13312608865377057, 0.13249682533512472, 0.13090499096660005, 0.13241528837225602, 0.1306094356126]
Avg. inf time for 'standard': inf
----------------------------------------
Avg. error for 'tree': 0.12786 (m) for the invidual flights: [0.1290971596